# Token Classification
This includes any problem that can be formulated as "attributing a label to each token in a sentence," eg
- Named Entity Recognition (NER)
- Part-of-speech tagging (POS)
- Chunking: finding the tokens that belong to the same entity

In [1]:
import os
import sys


# Connect to google drive
from google.colab import drive
os.chdir("/content")
drive.mount("/content/gdrive")

# Load colab_utils funtions
sys.path.append(f"/content/gdrive/MyDrive/repos/colab-utils")
import colab_utils

colab_utils.load_env_vars()
colab_utils.git_set_config()

PARENT_FOLDER = "/content/gdrive/MyDrive/repos"
os.chdir(PARENT_FOLDER)

git_repo = 'trevorki/huggingface-nlp' # replace with actual values
colab_utils.git_clone_repo(git_repo)

REPO_FOLDER = f"{PARENT_FOLDER}/{git_repo.split('/')[1]}"
os.chdir(REPO_FOLDER)

!pip install -r requirements.txt

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
# To use Huggingface hub on google colab
!git lfs install
!git lfs track "*.psd"
!git add .gitattributes

Updated git hooks.
Git LFS initialized.
Tracking "*.psd"


## Dataset
Use the `CoNLL-2003 dataset`, which contains news stories from Reuters. It contains labels for the three tasks we mentioned earlier: NER, POS, and chunking.

A big difference from other datasets is that the input texts are not presented as sentences or documents, but lists of words (the last column is called tokens, but it contains words in the sense that these are pre-tokenized inputs that still need to go through the tokenizer for subword tokenization).



In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
# these are not tokens, but lists of words
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

### NER labels

In [5]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [6]:
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [7]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


### POS Labels

In [8]:
# raw_datasets["train"][0]["pos_tags"]

In [9]:
# pos_feature = raw_datasets["train"].features["pos_tags"]
# pos_names = pos_feature.feature.names
# print(pos_names)

In [10]:
# words = raw_datasets["train"][0]["tokens"]
# labels = raw_datasets["train"][0]["pos_tags"]
# line1 = ""
# line2 = ""
# for word, label in zip(words, labels):
#     full_label = pos_names[label]
#     max_length = max(len(word), len(full_label))
#     line1 += word + " " * (max_length - len(word) + 1)
#     line2 += full_label + " " * (max_length - len(full_label) + 1)

# print(line1)
# print(line2)

### Chunking labels

In [11]:
# raw_datasets["train"][0]["chunk_tags"]

In [12]:
# chunk_feature = raw_datasets["train"].features["chunk_tags"]
# chunk_names = chunk_feature.feature.names
# print(chunk_names)

In [13]:
# words = raw_datasets["train"][0]["tokens"]
# labels = raw_datasets["train"][0]["chunk_tags"]
# line1 = ""
# line2 = ""
# for word, label in zip(words, labels):
#     full_label = chunk_names[label]
#     max_length = max(len(word), len(full_label))
#     line1 += word + " " * (max_length - len(word) + 1)
#     line2 += full_label + " " * (max_length - len(full_label) + 1)

# print(line1)
# print(line2)

# Tokenize
Since the tokenizer splits some words into multiple tokens we have to keed track of the `word_ids` to map them to their words later.

In [14]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(f"tokens: {inputs.tokens()}")
print(f"word_ids: {inputs.word_ids()}")

tokens: ['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']
word_ids: [None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]


Since the labels don't have the special characters or the works split into multiple tokens, we must align the tokenized inputs with the labels.
To

In [16]:
print(f"{len(raw_datasets['train'][0]['ner_tags'])} dataset labels:\n\t{raw_datasets['train'][0]['ner_tags']}")
print(f"{len(inputs.tokens())} tokenized inputs:\n\t{inputs.tokens()}")

9 dataset labels:
	[3, 0, 7, 0, 0, 0, 7, 0, 0]
12 tokenized inputs:
	['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']


In [17]:
def align_labels_with_tokens(labels, word_ids):
    """Adds items to labels to make them match the length of word_ids, by
    - duplicating the labels for tokens that were split from the same word
    - giving a label of -100 for all special tokens (so that it is ignored by coss entropy loss
    RETURNS:
        list: the new labels"""
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1 # this changes it from B to I
            new_labels.append(label)

    return new_labels

In [18]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [19]:
# Tokenize and align labels for whole dataset (We will pad it later)
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],
                                 truncation=True,
                                 is_split_into_words=True)
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [20]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

# Collating data
We can't use `DataCollatorWith Padding` because we must now pad both the inputs AND the labels, so we must use `DataCollatorForTokenClassification`, which takes the tokenizer as a parameter.

In [21]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer,
                                                   return_tensors="tf")

In [22]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"].numpy()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


array([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0,
        -100],
       [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100,
        -100]])

In [23]:
# Compare to just the labels
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


# Build TF Dataset

In [24]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

# The Model

In [25]:
!pwd

/content/gdrive/MyDrive/repos/huggingface-nlp


In [26]:
from transformers import TFAutoModelForTokenClassification

# translation dictionaries
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

Note also that we don’t supply a loss argument to compile(). This is because the models can actually compute loss internally — if you compile without a loss and supply your labels in the input dictionary (as we do in our datasets), then the model will train using that internal loss, which will be appropriate for the task and model type you have chosen.

In [28]:

# from transformers.keras_callbacks import PushToHubCallback

# callback = PushToHubCallback(output_dir="bert-finetuned-ner", tokenizer=tokenizer)

from datetime import datetime
print(datetime.now())

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    # callbacks=[callback],
    epochs=num_epochs,
    verbose=1
)
print(datetime.now())

2024-02-11 23:29:25.875930
Epoch 1/3
878/878 [==============================] - 213s 212ms/step - loss: 0.1720 - val_loss: 0.0652
Epoch 2/3
878/878 [==============================] - 198s 225ms/step - loss: 0.0484 - val_loss: 0.0509
Epoch 3/3
878/878 [==============================] - 197s 225ms/step - loss: 0.0283 - val_loss: 0.0505
2024-02-11 23:39:50.912547


In [33]:
from huggingface_hub import Repository
from huggingface_hub import create_repo


model_name = "bert-finetuned-ner"
hf_user = "Roverto"
model_folder = f"models/{model_name}"


create_repo(f"{model_name}", token = os.environ["HF_TOKEN"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Roverto/bert-finetuned-ner into local empty directory.


In [46]:
# save model and tokenizer to folder
model.save_pretrained(model_folder)
tokenizer.save_pretrained(model_folder)

('models/bert-finetuned-ner/tokenizer_config.json',
 'models/bert-finetuned-ner/special_tokens_map.json',
 'models/bert-finetuned-ner/vocab.txt',
 'models/bert-finetuned-ner/added_tokens.json',
 'models/bert-finetuned-ner/tokenizer.json')

In [47]:

# push model and tokenizer to repo
from huggingface_hub import HfApi
api = HfApi()

# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path=model_folder,
    repo_id=f"{hf_user}/{model_name}",
    repo_type="model",
    # commit_message  = "" #  Defaults to: f"Upload {path_in_repo} with huggingface_hub"
)

tf_model.h5:   0%|          | 0.00/431M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Roverto/bert-finetuned-ner/commit/c293db95e81731200a1ce271f7db6a03b891af76', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c293db95e81731200a1ce271f7db6a03b891af76', pr_url=None, pr_revision=None, pr_num=None)